# Principal Component Analysis (PCA)

Principal component analysis (PCA) is a method that reduces the dimensionality of a dataset while retaining most of its variation. It accomplishes this by identifying directions, called principal components, along which the variation in the data is maximal. By using a few components, each sample can be represented by relatively few numbers instead of by values for thousands of variables. Samples can then be plotted, making it possible to visually assess similarities and differences between samples and determine whether samples can be grouped.<sup><a href="#ref1">1</a></sup>

The image below<sup><a href="#ref2">2</a></sup> illustrates the principal components of a 2-dimensional dataset. $u_{1}$, the direction of greatest variation, is the first principal component, and $u_{2}$, perpendicular to $u_{1}$, the direction of the second most variation, is the second principal component:

<img src="http://datasets.genepattern.org/images/pca-vidal.png">

## 1. Requirements and considerations

- **Input datasets**
    - A dataset in the [GCT](https://www.genepattern.org/file-formats-guide#GCT) format. 
    - (Optional) A mapping of the samples in the dataset to classes in the [CLS](http://software.broadinstitute.org/cancer/software/genepattern/file-formats-guide#CLS) file format.
    
- **Output datasets**
    - the **principal component matrix** of the dataset (*u matrix*). This is the original dataset transformed into the coordinates of the axes of the principal components (eigenvectors).
    - the **principal component axes** (eigenvectors, *t matrix*). The first PC axis corresponds to the direction of greatest variation in the dataset. The following axes are the directions of each subsequent PC, in the order of decreasing variation explained.  
    - the **variance explained matrix** (eigenvalues, *s matrix*): Each entry in this matrix represents the amount of variance explained by its corresponding principal component.
 

## 2. Run PCA in GenePattern

### 2.1 Sign in to GenePattern

<div class="alert alert-info">
<h3 style="margin-top: 0;"> Instructions <i class="fa fa-info-circle"></i></h3>

- If you haven&#39;t yet logged in, enter your credentials into the cell below and click Login.
</div>

In [ ]:
# Requires GenePattern Notebook: pip install genepattern-notebook
import gp
import genepattern

# Username and password removed for security reasons.
genepattern.display(genepattern.session.register("https://cloud.genepattern.org/gp", "", ""))

### 2.2. Run PCA in GenePattern to compute the principal components of the dataset

<div class="alert alert-info">
<h3 style="margin-top: 0;"> Instructions <i class="fa fa-info-circle"></i></h3>

In the cell below:
- For **input dataset** upload or click and drag a file in GCT format. *Example*: [BRCA_HUGO_symbols.preprocessed.gct](https://datasets.genepattern.org/data/ccmi_tutorial/2017-12-15/BRCA_HUGO_symbols.preprocessed.gct),
- For the **cluster by** parameter, select whether you want to compute the principal components of the rows or the columns. Usually, columns represent samples and rows represent features of each sample (e.g., genes).
- Click **Run**.
- **NOTE**: you may see a warning message regarding the use of Pandas. You can ignore this warning.

</div>

In [ ]:
import re
import time
import numpy as np
import pandas as pd
import genepattern
import gp
import gp.data as gpdata
import nbtools
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import plotly
import plotly.graph_objs as go

class PCAMetadata:
    def __init__(self):
        return
    
pca_metadata = PCAMetadata()

@genepattern.build_ui(name="Compute Principal Components", 
                      description="Compute pcas, eigenvectors, and eigenvalues of a matrix in gct format", 
                      parameters={
                        "input_gct_fname": {"name":"input dataset","type": "file","kinds" : ["gct"], "optional":False},
                        "pca_axis": {"name":"Compute PCs of","type":"choice","choices":{"rows":1,"columns":3}, "default":3,"optional":False},
                        "output_var": {"hide": True}
                    })

def compute_principal_components(input_gct_fname, pca_axis):
    gct_fname = upload_input_file(input_gct_fname)
    
    pca_input_gct_df = gpdata.GCT(nbtools.open(gct_fname))
    pca_input_gct_df.reset_index(level="Description", inplace=True)
    pca_input_gct_df.reset_index(level="Name", inplace=True)

    pca_axis_str = {1:"rows",3:"columns"}[pca_axis]
    
    pca_metadata.sample_names = list(pca_input_gct_df.columns[2:])
    pca_metadata.feature_names = list(pca_input_gct_df["Name"])
    pca_metadata.feature_descriptions = list(pca_input_gct_df["Description"])
    pca_metadata.feature_annotations = [str(name) + "\n" + str(desc) if name != desc else name 
                                        for (name,desc) in zip(pca_metadata.feature_names,pca_metadata.feature_descriptions)]
    pca_metadata.pca_axis_str = pca_axis_str
    
    gpserver = genepattern.session.get(0)
        
    pca_module = gp.GPTask(gpserver, 'urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00017')
    pca_job_spec = pca_module.make_job_spec()
    
    if not is_url(input_gct_fname):
        gp_server_file = gpserver.upload_file(gct_fname, gct_fname)
        pca_job_spec.set_parameter("input.filename", gp_server_file.get_url())
    else:
        pca_job_spec.set_parameter("input.filename", input_gct_fname)

    pca_job_spec.set_parameter("cluster.by", pca_axis)
    pca_job_spec.set_parameter("output.file", "<input.filename_basename>")
    
    job = gpserver.run_job(pca_job_spec, False)
    
    uio = nbtools.UIOutput()
    status_bar=['-','\\','|','/']
    status_bar_pos = 0            
    genepattern.display(uio)
    while not job.is_finished():
        uio.status="Status:"+"<br>"+job.get_status_message()+'<br>'+status_bar[status_bar_pos]
        status_bar_pos = (status_bar_pos + 1) % len(status_bar)
        time.sleep(1)
        
    uio.close()
    
    file_list = [gct_fname]
    for f in job.output_files:
        file_list.append(f['link']['href'])
    
    uio = nbtools.UIOutput(files=file_list)
    genepattern.display(uio)
    
def upload_input_file(input_filename):
    if is_url(input_filename):
        with nbtools.open(input_filename) as f_in:
            input_file = f_in.read()

        output_filename = get_filename_from_url(input_filename)

        with open(output_filename, "wb") as f_out:
            f_out.write(input_file)
    else:
        output_filename = input_filename
    
    return(output_filename)

def get_filename_from_url(url):
    filename = re.sub(".*\/","",url)
    return(filename)

def is_url(path_or_url):
    
    url_re = re.compile(
            r'^(?:http|ftp)s?://'  # http:// or https://
            r'(?:(?:[A-Z0-9](?:[A-Z0-9-]{0,61}[A-Z0-9])?\.)+(?:[A-Z]{2,6}\.?|[A-Z0-9-]{2,}\.?)|'  # domain...
            r'localhost|'  # localhost...
            r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})'  # ...or ip
            r'(?::\d+)?'  # optional port
            r'(?:/?|[/?]\S+)$', re.IGNORECASE)

    if re.match(url_re, path_or_url):
        return(True)
    else:
        return(False)


When the job completes, you will see a new panel above with the title **Python Results**. You will also see 4 file links. Here `<filename>` is the name of the input file without its extension:

Filename | Description
:------------ | :-------------
`<filename>.gct` | the **input dataset** - the original dataset provided to the PCA analysis
`<filename>_u.odf` | the **principal components (u matrix)** - the original dataset, transformed into the coordinate system of the eigenvectors
`<filename>_t.odf` | the **eigenvectors (t matrix)** - the axes of the coordinate system oriented toward the directions of highest variability in the dataset
`<filename>_s.odf` | the **eigenvalues (s matrix)** - each entry in this matrix contains the amount of variance explained by the corresponding principal component

## 3. Visualize PCA results

We will visualize the first 3 principal components of the dataset. These correspond to the 3 axes of highest variance in the dataset, and when combined with a class labeling, can be used to determine how well-separated different classes of data are in the space of the principal components.

<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>

- Click **Run** in the cell below to load and initialize required packages
</div>

In [ ]:
# Allow embedding of plots in the output cell of the notebook
#plotly.offline.init_notebook_mode() 

# Enable embedding matplotlib graphs in the notebook:
%matplotlib inline

@genepattern.build_ui(name="Load Required Libraries", 
                      description="Load requried libraries and initialize plotting packages", 
                      parameters={
                        "output_var":{"hide":True}
                    })

def setup():
    print("Libraries loaded")
    
#
# Convert GPFile object to Numpy array
#

def gp_matrix_odf_to_nparray(gp_file):
    
    # Get a reference to your session with the GP server
    gpserver = genepattern.session.get(0)

    # Create the GPFile object (using the right credentials)
    fh = gp.GPFile(gpserver, gp_file)
    
    matrix_bytes = fh.read()

    # Remove header lines
    matrix_string = re.sub(".*\n", '', matrix_bytes, count=5, flags=0)
    matrix_string = re.sub("\t\n", '\n', matrix_string, count=0, flags=0)

    # The final split leaves an extra line, which must be removed
    matrix_list = matrix_string.split('\n')
    matrix_list.pop(len(matrix_list)-1)

    matrix_2dlist = [row.split('\t') for row in matrix_list]

    # Populate the new array with contents of the list:
    matrix_array = np.empty(shape=(len(matrix_2dlist),len(matrix_2dlist[0])))
 
    for r in range(len(matrix_2dlist)):
        for c in range(len(matrix_2dlist[0])):
            matrix_array[r][c] = matrix_2dlist[r][c]
            
    return(matrix_array)


###  3.1. Display scatter plot of first 3 principal components

<div class="alert alert-info">
<h3 style="margin-top: 0;"> Instructions <i class="fa fa-info-circle"></i></h3>

 In the cell below:
- Click the down arrow on the **gp u matrix file** parameter and select the file with the `_u.odf` suffix.
- (Optional) Upload or click and drag the file containing the phenotype descriptions to the **cls file url** parameter below. *Example*: [BRCA_HUGO_symbols.preprocessed.cls](https://datasets.genepattern.org/data/ccmi_tutorial/2017-12-15/BRCA_HUGO_symbols.preprocessed.cls)
- (Optional) add a title for the principal components graph into the **plot title** input. *Example*: `TCGA Breast Cancer vs. Normal`
- After clicking **Run**, you will see a 3D scatter plot of the first 3 principal components of the dataset. To explore the dataset:
    - Click and drag to rotate left, right, up, or down
    - Scroll to zoom in and out
    - Hover over a point to see its data value.
    - Hover over the upper right hand side of the graph to see additional options such as image export.
</div>

In [ ]:
@genepattern.build_ui(name="Plot Principal Components", 
                      description="Read PCA result files and class assignments into Python variables", 
                      parameters={
                        "gp_u_matrix_file":{"type": "file", "kinds": ["_u.odf"]},
                        "input_cls_fname": {"name":"input class file","type": "file","kinds" : ["cls"], "default":"","optional":True},
                        "plot_title":{"type":"string", "default":"Principal Components", "optional":True},
                        "output_var": {"default":"pca_data", "hide": True}
                    })

def plot_principal_components(gp_u_matrix_file, input_cls_fname, plot_title):
    
    if (len(input_cls_fname) != 0):
        cls_fname = upload_input_file(input_cls_fname)
        cls_info = gpdata.CLS(cls_fname)
        pca_metadata.class_assignments = cls_info.class_assignments
        pca_metadata.class_names = cls_info.class_names
        pca_metadata.num_classes = cls_info.num_classes
    else:
        pca_metadata.class_names=[]
        pca_metadata.num_classes=1
        pca_metadata.class_assignments=[]

    (sample_names, feature_annotations, pca_axis_str, num_classes, class_assignments, class_names) = (pca_metadata.sample_names,
                                                     pca_metadata.feature_annotations,
                                                     pca_metadata.pca_axis_str,
                                                     pca_metadata.num_classes,
                                                     pca_metadata.class_assignments,
                                                     pca_metadata.class_names)

    u_matrix = gp_matrix_odf_to_nparray(gp_u_matrix_file)

    # The principal components are the transpose of the u matrix:
    pc = u_matrix.transpose()
    
    # Create color map for up to 23 classes, corresponding to the rainbow_discrete colormap
    # adapted from Python code available at https://personal.sron.nl/~pault/
    
    clrs = ['#E8ECFB', '#D9CCE3', '#D1BBD7', '#CAACCB', '#BA8DB4', '#AE76A3',
            '#AA6F9E', '#994F88', '#882E72', '#1965B0',  '#437DBF', '#5289C7',
            '#6195CF', '#7BAFDE', '#4EB265', '#90C987', '#CAE0AB', '#F7F056',
            '#F7CB45', '#F6C141', '#F4A736', '#F1932D', '#EE8026', '#E8601C',
            '#E65518','#DC050C', '#A5170E', '#72190E', '#42150A']

    indexes = [[9],
               [9, 25], 
               [9, 17, 25], 
               [9, 14, 17, 25], 
               [9, 13, 14, 17, 25], 
               [9, 13, 14, 16, 17, 25], 
               [8, 9, 13, 14, 16, 17, 25], 
               [8, 9, 13, 14, 16, 17, 22, 25],
               [8, 9, 13, 14, 16, 17, 22, 25, 27],
               [8, 9, 13, 14, 16, 17, 20, 23, 25, 27],
               [8, 9, 11, 13, 14, 16, 17, 20, 23, 25, 27],
               [2, 5, 8, 9, 11, 13, 14, 16, 17, 20, 23, 25],
               [2, 5, 8, 9, 11, 13, 14, 15, 16, 17, 20, 23, 25],
               [2, 5, 8, 9, 11, 13, 14, 15, 16, 17, 19, 21, 23, 25],
               [2, 5, 8, 9, 11, 13, 14, 15, 16, 17, 19, 21, 23, 25, 27],
               [2, 4, 6, 8, 9, 11, 13, 14, 15, 16, 17, 19, 21, 23, 25, 27],
               [2, 4, 6, 7, 8, 9, 11, 13, 14, 15, 16, 17, 19, 21, 23, 25, 27],
               [2, 4, 6, 7, 8, 9, 11, 13, 14, 15, 16, 17, 19, 21, 23, 25, 26, 27],
               [1, 3, 4, 6, 7, 8, 9, 11, 13, 14, 15, 16, 17, 19, 21, 23, 25, 26, 27],
               [1, 3, 4, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17, 19, 21, 23, 25, 26, 27],
               [1, 3, 4, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17, 18, 20, 22, 24, 25, 26, 27],
               [1, 3, 4, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17, 18, 20, 22, 24, 25, 26, 27, 28],
               [0, 1, 3, 4, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17, 18, 20, 22, 24, 25, 26, 27, 28]]

    data=[]
    traces=[]
    label_list = sample_names if pca_axis_str == 'columns' else feature_annotations

    #
    # If a cls file is provided, obtain the colormap for the given number of classes.
    # Otherwise, use the color in the 1-class colormap:
    #
    
    if num_classes > 1:
       # Determine appropriate map for the given number of classes:
        colormap = [clrs[i] for i in indexes[num_classes-1]]
        colors = [colormap[class_assignments[i]] for i in range(len(class_assignments))]
        
        for j in range(num_classes):
            traces.append(go.Scatter3d(
            x=[i for indx,i in enumerate(pc[0]) if class_assignments[indx] == j],
            y=[i for indx,i in enumerate(pc[1]) if class_assignments[indx] == j],
            z=[i for indx,i in enumerate(pc[2]) if class_assignments[indx] == j],
            text = [i for indx,i in enumerate(label_list) if class_assignments[indx]==j],
            mode='markers',
            name=class_names[j],
            marker=go.scatter3d.Marker(size=10,color=colormap[j])))
            data.append(traces[j])
    else:
        colormap = [clrs[indexes[0][0]]]
        colors = [colormap[0] for i in range(len(pc[0]))]
    
        traces.append(go.Scatter3d(
        x=pc[0],
        y=pc[1],
        z=pc[2],
        mode='markers',
        text = label_list,
        marker=go.scatter3d.Marker(size=10,color=colormap[0])))
        data.append(traces[0])

    layout=go.Layout(dict(title=plot_title,
                scene=dict(xaxis=dict(title="PC 1",visible=True),
                yaxis=dict(title="PC 2",visible=True),
                zaxis=dict(title="PC 3",visible=True))
                )
    )

    fig=dict(data=traces, layout=layout)
    plotly.offline.iplot(fig)
    
    return(pc)


### 3.2. Display percentage of variance explained for each principal component

The **variance explained** graph provides an idea of how many principal components are required to represent the majority of the dataset. You can use this to select the number of principal components to use to represent your data.

<div class="alert alert-info">
<h3 style="margin-top: 0;"> Instructions <i class="fa fa-info-circle"></i></h3>
    
- For the **eigenvalues file**, click the down arrow in the input box and select the file with the `_s.odf` suffix.
- After clicking **Run**, you will see plots of **variance explained** and **cumulative variance explained**:
    - The bar chart represents the variance explained for each principal component.
    - The line graph shows the cumulative variance explained.
</div>

In [ ]:
@genepattern.build_ui(name="Plot Variance Explained", 
                      description="Plot variance of each principal component", 
                      parameters={
                          "eigenvalues_file":{"type": "file", "kinds": ["_s.odf"]},
                          "output_var": {"hide": True}
                    })

def plot_variance_explained(eigenvalues_file):
    
    s_matrix = gp_matrix_odf_to_nparray(eigenvalues_file)
        
    # Convert eigenvalues from an array to a list
    # The eigenvalue matrix only has entries on the diagonal. Extract these into a list to facilitate processing:
    evalues = [s_matrix[x][x] for x in range(len(s_matrix))]
    
    # Compute percentage contribution of each eigenvector
    ev_total = sum(evalues)
    ev_percents = evalues/ev_total

    bar_offset_x = -0.25
    bar_offset_y = 0.01
    plt.figure(figsize=(15,10))
    plt.clf()

    x_ticks = [str(i+1) for i in range(len(ev_percents))]

    #
    # Plot the variance explained of each PC:
    #
    
    plt.title("Variance Explained Per Principal Component")
    plt.xlabel("Principal Component")
    plt.ylabel("Variance Explained")

    plt.bar(x_ticks, ev_percents, 0.8)
    for i in range(len(ev_percents)):
        plt.text(i + bar_offset_x, ev_percents[i]+ bar_offset_y, pct_format(ev_percents[i]))


    #
    # Plot cumulative variance explained:
    #

    cum_total = [sum(ev_percents[0:i+1]) for i in range(len(ev_percents))]
    plt.plot(x_ticks, cum_total, "go-")
    for i in range(len(cum_total)):
        plt.text(i + bar_offset_x, cum_total[i]+ bar_offset_y, pct_format(cum_total[i]))
        
    plt.legend(["Cumulative total variance","Variance explained per PC"],loc='best')
 
    plt.show()
    return

def pct_format(val):
    res = str(int(val * 100))
    return(res)


## 4. Export principal components

To use the principal components in further analyses, you can export the dataset.

<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>

- **Number of pcs**: enter the number of principal components in the dataset to export
- **File type**: select the format of the output dataset. If you want to run additional GenePattern analyses on the principal components, select GCT. If you want to run analyses in other tools, select TXT.
- **File name**: enter an output file name or use the default
- Click **Run**
- You will see a panel containing a link to the exported file. Click the <i class="fa fa-info-circle"></i> next to the filename to view the options available for the file.
</div>

In [ ]:
@genepattern.build_ui(name="Export PCA dataset", 
                      description="Save the PCA dataset", 
                      parameters={
                        "num_pcs": {"name":"Number of PCs","type": "string", "default":"3", "optional":False},
                        "file_name":{"type": "string", "default":"pca_output","optional":False},
                        "file_type":{"type": "choice", "optional":False,
                                    "choices":{"GCT":"gct",
                                               "TXT (tab-delimited)":"txt"
                                              },
                                    "default":"gct"},
                        "output_var": {"hide": True}
                    })

def export_pca_dataset(num_pcs, file_name, file_type):
    pcs = pca_data
    num_rows = len(pcs[0])
    num_pcs_re = re.compile("^[1-9][0-9]*$")
    
    num_pcs_match = re.match(num_pcs_re, str(num_pcs))
    if not num_pcs_match:
        raise ValueError("Number of PCs parameter must be a positive integer")
    
    num_cols = num_pcs if num_pcs <= len(pcs) else len(pcs)
    
    if not((file_name).endswith(".gct") or (file_name).endswith(".txt")):
        file_name += "."+file_type
    
    out_f = open(file_name, "w")

    if file_type == "gct":
        out_f.write("#1.2\n")
        out_f.write(str(num_rows)+"\t"+str(num_cols)+"\n")
        name_line = "Name\tDescription"+"\t"+"\t".join(["pc_"+ str(i+1) for i in range(num_cols)])+"\n"
        out_f.write(name_line)
        
    feature_names = pca_metadata.sample_names if pca_metadata.pca_axis_str == "columns" else pca_metadata.feature_names
    feature_descriptions = pca_metadata.sample_names if pca_metadata.pca_axis_str == "columns" else pca_metadata.feature_descriptions

    for line in range(num_rows):
        if(file_type == "gct"):
            out_f.write(feature_names[line]+"\t"+feature_descriptions[line]+"\t")
        value_list = "\t".join([str(pcs[line][i]) for i in range(num_cols)])
        out_f.write(value_list)
        out_f.write("\n")
    out_f.close()

    uio = nbtools.UIOutput(files=[file_name])
    genepattern.display(uio)


## References

<a name="ref1">1</a>. Ringnér, M. What is principal component analysis?. _Nat Biotechnol_ **26**, 303–304 (2008). https://doi.org/10.1038/nbt0308-303

<a name="ref2">2</a>. Vidal, Rene, Yi Ma, and Shankar Sastry. "Generalized principal component analysis (GPCA)." _IEEE transactions on pattern analysis and machine intelligence_ 27.12 (2005): 1945-1959. https://doi.org/10.1109/tpami.2005.244